# QKeras Model #1

In [1]:
import pandas as pd

In [6]:
# Load data for our target gate family (rotations about Bloch sphere x-axis)
df = pd.read_csv("../data/xrotate.csv", header=None)

In [7]:
df.drop(0, inplace=True) # Remove row for beta=-pi (same alpha as beta=+pi)
df.drop(50, inplace=True) # Remove row for beta=0

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
1,-0.002227,-0.013359,-0.017554,-0.015438,-0.017162,-0.017211,-0.015443,-0.017591,-0.013627,-0.001845,...,0.000072,0.000049,0.000033,0.000031,-0.000007,0.000022,0.000083,1.993576e-05,0.000100,-3.078761
2,-0.003243,-0.014217,-0.016003,-0.015864,-0.015976,-0.015975,-0.015850,-0.016015,-0.014253,-0.003144,...,0.000021,0.000005,-0.000008,-0.000011,-0.000030,-0.000011,0.000028,1.926120e-06,0.000052,-3.015929
3,-0.002533,-0.013236,-0.015930,-0.015621,-0.015832,-0.015857,-0.015657,-0.015920,-0.013301,-0.002484,...,-0.000042,-0.000021,0.000003,0.000015,0.000036,0.000008,-0.000044,-2.058282e-05,-0.000058,-2.953097
4,-0.003137,-0.013546,-0.015116,-0.015012,-0.015091,-0.015093,-0.015003,-0.015126,-0.013592,-0.003011,...,0.000029,0.000011,-0.000001,-0.000003,-0.000029,-0.000007,0.000039,2.099520e-08,0.000067,-2.890265
5,-0.003040,-0.013286,-0.014874,-0.014792,-0.014853,-0.014853,-0.014784,-0.014882,-0.013323,-0.002937,...,0.000026,0.000010,-0.000001,-0.000003,-0.000029,-0.000008,0.000036,-3.342383e-06,0.000065,-2.827433


In [9]:
df.shape

(99, 21)

In [11]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, Y_train, Y_test = train_test_split(df.iloc[:,20], df.iloc[:,:20], test_size=0.3, random_state=5)

In [14]:
import qkeras
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

In [16]:
# Build quantized model
model = tf.keras.models.Sequential()
model.add(QDense(4, input_shape=(1,), name='fc1',
                 kernel_quantizer=quantized_bits(14,2,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(14,2,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))
model.add(QActivation(activation=quantized_relu(14,2,qnoise_factor=1.0), name='relu1'))
model.add(QDense(8, name='fc2e',
                 kernel_quantizer=quantized_bits(12,1,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(12,2,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))
model.add(QActivation(activation=quantized_relu(12,2,qnoise_factor=1.0), name='relu2e'))
model.add(QDense(11, name='fc3',
                 kernel_quantizer=quantized_bits(10,0,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(10,0,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))
model.add(QActivation(activation=quantized_relu(12,2,qnoise_factor=1.0), name='relu3'))
model.add(QDense(11, name='fc3e',
                 kernel_quantizer=quantized_bits(10,1,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(10,2,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))
model.add(QActivation(activation=quantized_relu(10,2,qnoise_factor=1.0), name='relu3e'))
model.add(QDense(11, name='fc3e2',
                 kernel_quantizer=quantized_bits(10,1,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(10,0,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))
model.add(QActivation(activation=quantized_relu(10,2,qnoise_factor=1.0), name='relu3e2'))
model.add(QDense(11, name='fc3e4',
                 kernel_quantizer=quantized_bits(10,1,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(10,0,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))
model.add(QActivation(activation=quantized_relu(10,2,qnoise_factor=1.0), name='relu3e4'))
model.add(QDense(11, name='fc3e3',
                 kernel_quantizer=quantized_bits(10,1,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(10,0,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))
model.add(QActivation(activation=quantized_relu(10,2,qnoise_factor=1.0), name='relu3e3'))
model.add(QDense(20, name='output',
                 kernel_quantizer=quantized_bits(12,0,alpha=1,qnoise_factor=1.0), bias_quantizer=quantized_bits(12,0,alpha=1,qnoise_factor=1.0),
                 kernel_initializer='lecun_uniform'))

In [17]:
from callbacks import all_callbacks

In [25]:
train = False # Set this to True to retrain the model. Otherwise, load trained model.
if train:
    model.compile(optimizer=tf.optimizers.Adam(lr=0.001),
                  loss=tf.losses.MeanSquaredError(),
                  metrics=[
                      tf.metrics.MeanSquaredLogarithmicError(),
                      tf.metrics.MeanSquaredError(),
                      tf.metrics.MeanAbsolutePercentageError()
                  ])
    callbacks = all_callbacks(stop_patience=1000,
                                lr_factor=0.5,
                                lr_patience=100,
                                lr_epsilon=0.000001,
                                lr_cooldown=2,
                                lr_minimum=0.0000001,
                                outputDir="../model/training")
    model.fit(x_train, Y_train, epochs=5000, validation_split=0.30, callbacks=callbacks.callbacks, verbose=1)
    model.save("../model/qmodel.h5")
else:
    co = {}
    qkeras.utils._add_supported_quantized_objects(co)
    model = tf.keras.models.load_model("../model/qmodel.h5", custom_objects=co)

In [24]:
model.evaluate(x_test, Y_test)

1/1 [==============================] - 0s 42ms/step - loss: 9.6363e-08 - mean_squared_logarithmic_error: 7.1856e-08 - mean_squared_error: 9.6363e-08 - mean_absolute_percentage_error: 59.1056


[9.636322317874146e-08,
 7.185615658045208e-08,
 9.636322317874146e-08,
 59.10564041137695]